In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
stores = pd.read_csv("/content/stores.csv")
transactions = pd.read_csv("/content/transactions.csv")
oil = pd.read_csv("/content/oil.csv")
holidays_events = pd.read_csv("/content/holidays_events.csv")

In [3]:
# Calculating rolling percentage change
oil['oil_pct_change_rolling'] = oil['dcoilwtico'].pct_change().rolling(7).mean()

/tmp/ipython-input-2134042104.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  oil['oil_pct_change_rolling'] = oil['dcoilwtico'].pct_change().rolling(7).mean()


In [4]:
oil.head(10)

,date,dcoilwtico,oil_pct_change_rolling
0,2013-01-01,NaN,NaN
1,2013-01-02,93.14,NaN
2,2013-01-03,92.97,NaN
3,2013-01-04,93.12,NaN
4,2013-01-07,93.20,NaN
5,2013-01-08,93.21,NaN
6,2013-01-09,93.08,NaN
7,2013-01-10,93.81,NaN
8,2013-01-11,93.60,0.000709
9,2013-01-14,94.27,0.001992


In [5]:
holidays_events = pd.get_dummies(holidays_events, columns=['type', 'locale', 'locale_name'])

# Creating binary flag for transferred holidays
holidays_events['is_transferred'] = holidays_events['transferred'].astype(int)

# Creating binary flags for pre and post holiday events
holidays_events['pre_holiday'] = holidays_events['description'].apply(lambda x: 1 if 'pre' in x.lower() else 0)
holidays_events['post_holiday'] = holidays_events['description'].apply(lambda x: 1 if 'post' in x.lower() else 0)

In [6]:
holidays_events.head()

,date,description,transferred,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,locale_Local,...,locale_name_Quevedo,locale_name_Quito,locale_name_Riobamba,locale_name_Salinas,locale_name_Santa Elena,locale_name_Santo Domingo,locale_name_Santo Domingo de los Tsachilas,is_transferred,pre_holiday,post_holiday
0,2012-03-02,Fundacion de Manta,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,0,0,0
1,2012-04-01,Provincializacion de Cotopaxi,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,0,0,0
2,2012-04-12,Fundacion de Cuenca,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,0,0,0
3,2012-04-14,Cantonizacion de Libertad,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,0,0,0
4,2012-04-21,Cantonizacion de Riobamba,False,False,False,False,True,False,False,True,...,False,False,True,False,False,False,False,0,0,0


In [7]:
transactions['date'] = pd.to_datetime(transactions['date'])

In [8]:
transactions = transactions.sort_values(['store_nbr', 'date'])

In [9]:
# Calculate rolling average transactions per store
transactions['transactions_rolling_avg'] = transactions.groupby('store_nbr')['transactions'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Calculate cumulative transactions per store
transactions['transactions_cumulative'] = transactions.groupby('store_nbr')['transactions'].cumsum()

In [10]:
transactions.head()

,date,store_nbr,transactions,transactions_rolling_avg,transactions_cumulative
1,2013-01-02,1,2111,2111.000000,2111
47,2013-01-03,1,1833,1972.000000,3944
93,2013-01-04,1,1863,1935.666667,5807
139,2013-01-05,1,1509,1829.000000,7316
185,2013-01-06,1,520,1567.200000,7836


In [11]:
print(stores.columns)

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')


In [12]:
# Convert store features into usable model inputs
stores = pd.get_dummies(stores, columns=['city', 'type', 'cluster'],dtype=int)
display(stores.head())

,store_nbr,state,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,...,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17
0,1,Pichincha,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,Pichincha,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,Pichincha,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,4,Pichincha,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,Santo Domingo de los Tsachilas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
train['date'] = pd.to_datetime(train['date'])

ValueError: time data "2013-" doesn't match format "%Y-%m-%d", at position 314. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
# Merge train with stores
train = pd.merge(train, stores, on='store_nbr', how='left')

In [14]:
# Merge train with oil
train = pd.merge(train, oil, on='date', how='left')

In [15]:
# Merge train with holidays_events
train = pd.merge(train, holidays_events, on='date', how='left')

In [16]:
# Merge train with transactions
train = pd.merge(train, transactions, on=['date', 'store_nbr'], how='left')

ValueError: You are trying to merge on object and datetime64[ns] columns for key 'date'. If you wish to proceed you should use pd.concat

In [19]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,state,city_Ambato,city_Babahoyo,city_Cayambe,...,locale_name_Quevedo,locale_name_Quito,locale_name_Riobamba,locale_name_Salinas,locale_name_Santa Elena,locale_name_Santo Domingo,locale_name_Santo Domingo de los Tsachilas,is_transferred,pre_holiday,post_holiday
0,0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,Pichincha,0.0,0.0,0.0,...,False,False,False,False,False,False,False,0.0,0.0,0.0
1,1,2013-01-01,1.0,BABY CARE,0.0,0.0,Pichincha,0.0,0.0,0.0,...,False,False,False,False,False,False,False,0.0,0.0,0.0
2,2,2013-01-01,1.0,BEAUTY,0.0,0.0,Pichincha,0.0,0.0,0.0,...,False,False,False,False,False,False,False,0.0,0.0,0.0
3,3,2013-01-01,1.0,BEVERAGES,0.0,0.0,Pichincha,0.0,0.0,0.0,...,False,False,False,False,False,False,False,0.0,0.0,0.0
4,4,2013-01-01,1.0,BOOKS,0.0,0.0,Pichincha,0.0,0.0,0.0,...,False,False,False,False,False,False,False,0.0,0.0,0.0


In [23]:
nan_counts = train.isnull().sum()
print("NaN counts per column in train DataFrame:")
print(nan_counts[nan_counts > 0])

NaN counts per column in train DataFrame:
oil_pct_change_rolling                         892782
description                                   2551824
transferred                                   2551824
type_Additional                               2551824
type_Bridge                                   2551824
type_Event                                    2551824
type_Holiday                                  2551824
type_Transfer                                 2551824
type_Work Day                                 2551824
locale_Local                                  2551824
locale_National                               2551824
locale_Regional                               2551824
locale_name_Ambato                            2551824
locale_name_Cayambe                           2551824
locale_name_Cotopaxi                          2551824
locale_name_Cuenca                            2551824
locale_name_Ecuador                           2551824
locale_name_El Carmen                   

In [19]:
train['dcoilwtico'] = train['dcoilwtico'].ffill().bfill()

In [20]:
train[['type', 'locale', 'locale_name', 'description']] = train[['type', 'locale', 'locale_name', 'description']].fillna('No Holiday')
train['transferred'] = train['transferred'].fillna(False)

KeyError: "['type', 'locale', 'locale_name'] not in index"

In [21]:
train['transactions'] = train['transactions'].fillna(0)

KeyError: 'transactions'

In [18]:
# Plot Sales vs Transactions
plt.figure(figsize=(10, 6))
sns.barplot(data=train, x='transactions', y='sales')
plt.title('Sales vs Transactions')
plt.xlabel('Transactions')
plt.ylabel('Sales')
plt.show()

ValueError: Could not interpret value `transactions` for `x`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>

In [ ]:
# Plot Sales vs Oil Price
plt.figure(figsize=(10, 6))
sns.barplot(data=train, x='dcoilwtico', y='sales')
plt.title('Sales vs Oil Price')
plt.xlabel('Oil Price (dcoilwtico)')
plt.ylabel('Sales')
plt.show()